# Import

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Implement

In [2]:
class MLPClassifier:
    def __init__(self, hidden_layer_sizes=[10], alpha=0.0001, batch=32, max_iter=1000, l2=0.01, tol = 1e-4, random_state=42):
        ''' 
        Class constructor
        We use the sigmoid activation in the hidden layers, and the softmax activation at the output.
        Parameters
        ----------
        alpha: the learning rate determines how big the step would be on each iteration.
        batch: Using batch samples for one times update weight
        max_iter: number of times update weight
        l2: l2 regularization
        tol: weight threshold changes to stopping.
        '''
        self.hidden_layer_sizes = hidden_layer_sizes
        self.alpha = alpha
        self.batch = batch
        self.max_iter = max_iter
        self.l2 = l2
        self.tol = tol
        self.random_state = random_state
        self.Ws = None
    
    def sigmoid(self, x):
        '''
        Computes sigmoid function for each element of array x.
        '''
        return 1 / (1 + np.exp(-x))
    
    def gradientSigmoid(self, As, Ws ,mb_X, delta, i):
        '''
        Computes gradient vector for sigmoid layer.
        '''
        temp = delta.copy()
        delta = []
        grad = []
        for j in range(i, 0, -1):
            temp = np.multiply(np.dot(temp, self.Ws[j].T), As[j] - np.power(As[j],2))
            temp = temp[:,1:]
            delta.append(temp)
            grad.append(np.dot(As[j-1].T,temp) / len(As[j-1]))
        return delta, grad
    
    
    def softmax(self, z):
        '''
        Computes softmax function for each row of array z.
        '''
        # Subtract the largest value in that column to fix overflow exp.
        A = np.exp(z - np.max(z, axis = 1, keepdims = True))
        A /= A.sum(axis = 1, keepdims=True)

        return A
    
    def gradientSoftmax(self, As, x, y):
        '''
        Computes gradient vector for softmax layer
        '''
        delta = As[-1] - y
        grad = (As[-2].T @ delta)/ len(x)
        return delta, grad

    def computeForwardPropagation(self, Ws, X, need_all_layer_outputs):
        '''
        Computes the outputs of Neural Net by forward propagating X through the network.
        '''
        num_layers = len(self.Ws)
        As = [X]
        a_1 = X
        for i in range(num_layers - 1):
            z = np.dot(a_1, self.Ws[i])  
            a = self.sigmoid(z)
            a = np.append(np.ones((a.shape[0], 1)), a, axis=1)
            As.append(a)
            a_1 = a
        z_last = np.dot(As[-1], self.Ws[-1])
        a_last = self.softmax(z_last)
        As.append(a_last)
        if need_all_layer_outputs:
            return As
        else:
            return As[-1]
    
    def computeLayerSizes(self, X, Y, hid_layer_sizes):
        num_classes = len(np.unique(Y)) # Num classes
        layer_sizes = [X.shape[1]] + hid_layer_sizes + [num_classes]
        return layer_sizes

    def oneHotEncoding(self, Y, num_classes):
        '''
        Y to one hot encoding
        
        Parameters
        ---------
        y: numpy array, shape (m, 1) 
        The vector of outputs.
        
        Return
        y: numpy array, shape (m, num_classes) 
        '''
        one_hot_Y = np.zeros((len(Y), num_classes))
        one_hot_Y[np.arange(len(Y)), Y.reshape(-1)] = 1
        return one_hot_Y
    
    def initWeight(self, X, Y, layer_sizes):
        np.random.seed(self.random_state) 
        self.Ws = np.array([np.random.randn(layer_sizes[i] + 1 , layer_sizes[i + 1]) / np.sqrt(layer_sizes[i] + 1) 
              for i in range(len(layer_sizes) - 1)]) 
    
    def updateWeights(self, Ws, As, mb_X, mb_Y, alpha):
        
        #update weights for softmax layer
        num_hidden_layer = len(self.Ws) - 1
        delta_last , grad_last = self.gradientSoftmax(As, mb_X, mb_Y)
        delta , grad = self.gradientSigmoid(As, self.Ws , mb_X, delta_last, num_hidden_layer)
        self.Ws[-1] -= alpha * (grad_last  + self.l2 * grad_last) / len(mb_X)
        
        #update weights for hidden layer
        grad = grad[::-1]
        for i in range(len(delta)):
            self.Ws[i] -= alpha * (grad[i] + self.l2 * grad[i]) / len(mb_X)

    def fit(self, X, y):
        '''
        Trains Softmax Regression on the dataset (X, y) using mini-batch Gradient Descent.
        
        Parameters
        ----------
        X : numpy array, shape (m, n)
        The matrix of inputs
        y : numpy array, shape (m, 1) 
        The vector of outputs.
        '''
        # Get layer sizes
        layer_sizes = self.computeLayerSizes(X, y, self.hidden_layer_sizes)
        
        # Prepare for training
        self.initWeight(X, y, layer_sizes)
        y = self.oneHotEncoding(y, layer_sizes[-1])
    
        # First column of this matrix is all ones (corresponding to x_0).
        X = np.append(np.ones((X.shape[0], 1)), X, axis=1)
        m, n = X.shape
        
        # Checking for changes in weight after 20 iter 
        check_w_after = 20
        
        for iter in range(1, self.max_iter + 1):
            # mix data 
            mix_id = np.random.permutation(m)
            for i in list(range(0, m, self.batch)):
                # Get batch samples
                mb_X = X[mix_id[i : i + self.batch]]
                mb_Y = y[mix_id[i : i + self.batch]]
                
                # Compute forward propagation (all layers)
                As = self.computeForwardPropagation(self.Ws, mb_X, True)
                
                # Back propagation, compute each layer's gradient and update its W
                self.updateWeights(self.Ws, As, mb_X, mb_Y, self.alpha)             
        
    def predict(self, X):
        '''
        Predict using the Softmax Regression model.
        
        Parameters
        ----------
        X : numpy array, shape (m, n)
        The matrix of inputs
        
        Return
        ----------
        Returns predicted values.
        '''
        # First column of this matrix is all ones (corresponding to x_0).
        X = np.append(np.ones((X.shape[0], 1)), X, axis = 1)    
        # Compute training info, save it, and print it
        A = self.computeForwardPropagation(self.Ws, X, False)
        return np.argmax((A), axis = 1).reshape(-1,1)

In [3]:
def standardScaler(X):
    return (X - np.mean(X)) / np.std(X)

# Test

In [4]:
iris = datasets.load_iris()

X = iris.data
y = iris.target.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
# Using 1 hidden layer with 50 units
model = MLPClassifier(hidden_layer_sizes=[50], alpha=0.2)

X_train = standardScaler(X_train)
model.fit(X_train, y_train)

X_test = standardScaler(X_test)
y_pred = model.predict(X_test)

In [6]:
# Using 1 hidden layer with 100 units
model = MLPClassifier(hidden_layer_sizes=[100], alpha=0.2)

model.fit(X_train, y_train)

y_pred1 = model.predict(X_test)

In [7]:
# Using 2 hidden layers with (50,50) units
model = MLPClassifier(hidden_layer_sizes=[50,50], alpha=0.2)

model.fit(X_train, y_train)

y_pred2 = model.predict(X_test)

In [8]:
# Using 2 hidden layers with (100,100) units
model = MLPClassifier(hidden_layer_sizes=[100,100], alpha=0.2)

model.fit(X_train, y_train)

y_pred3 = model.predict(X_test)

In [9]:
print('Score of using 1 hidden layer with 50 units:', np.mean(y_pred == y_test))
print('Score of using 1 hidden layer with 100 units:', np.mean(y_pred1 == y_test))
print('Score of using 2 hidden layers with (50,50) units:', np.mean(y_pred2 == y_test))
print('Score of using 2 hidden layers with (100,100) units:', np.mean(y_pred3 == y_test))

Score of using 1 hidden layer with 50 units: 0.9111111111111111
Score of using 1 hidden layer with 100 units: 1.0
Score of using 2 hidden layers with (50,50) units: 0.9333333333333333
Score of using 2 hidden layers with (100,100) units: 0.9333333333333333
